In [113]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
import librosa.display
from scipy.signal import find_peaks
import math
import os

In [105]:
# Cough audio loading
cough_audios = []
cough_timestamps = []

# Getting records and timestamp files
audio_file_count = len(os.listdir('./records'))

i = 1
while i < audio_file_count:
    cough_audios.append("./records/audio-{}.wav".format(i))
    cough_timestamps.append("./timestamps/audio-label-{}.txt".format(i))
    i += 1

# Loading audio files to python
audio_data = []

for cough_audio in cough_audios:
    # Target sample rate: 48000
    data, sample_rate = librosa.load(cough_audio, sr=48000)
    audio_data.append(data)

In [106]:
# Read timestamps from text files
real_timestamps = []

for cough_timestamp in cough_timestamps:
    
    timestamp = []
    f = open(cough_timestamp, "r")
    content = f.read()
    content = content.split("\n")

    for line in content:
        if line != "":
            timestamp.append(float(line.split("\t")[0]))
    
    real_timestamps.append(timestamp)
    
print(len(real_timestamps))

9


In [107]:
# Moving Average of the Data:
def compute_moving_average(data, window_size = 15):
    
    moving_averages = []

    i = 0
    while i < len(data) - window_size + 1:
        
        window = data[i : i + window_size]
        window_average = round(np.sum(window) / window_size, 2)
        moving_averages.append(window_average)

        i += 1    
    
    return np.array(moving_averages)

In [108]:
def normalize_data(data):
    min = np.min(data)
    max = np.max(data)

    data = (data - min) / (max - min)
    return data

In [109]:
# Getting moving averages of each audio file
moving_averages = [compute_moving_average(data) for data in audio_data]
normalized_moving_average_data = [normalize_data(data) for data in moving_averages]

In [110]:
# Finding the number of coughs with a threshold
all_timestamp_data = []

# Statistics of the data
for i_data, moving_avg_data in enumerate(normalized_moving_average_data):

    max_value = np.max(moving_avg_data)
    mean_value = np.mean(moving_avg_data)
    std = np.std(moving_avg_data)

    # Threshold 
    percentile_threshold = 99.8
    threshold = np.percentile(moving_avg_data, percentile_threshold)

    # Peak detection
    cough_indices, _ = find_peaks(moving_avg_data, prominence = 0.5)
    cough_indices = list(cough_indices)

    # Deleting overlaps in the peaks - Avoiding to count same cough more than one
    i = 0
    while i < len(cough_indices):

        peak = cough_indices[i]
        peak_range = (peak - 5000, peak + 5000) # The range is determined experimentally

        overlap_indices = [index for index in cough_indices
                       if peak_range[0] < index < peak_range[1]]

        if len(overlap_indices) > 1:

            # Find the index with maximum amplitude 
            max = overlap_indices[0]
            for index in overlap_indices:
                if (moving_avg_data[index] > moving_avg_data[max]):
                    max = index

            overlap_indices.remove(max)

            for element in overlap_indices:
                cough_indices.remove(element)

        i += 1


    # Applying the threshold
    cough_indices_copy = cough_indices.copy()
    for index in cough_indices_copy:

        amplitude = moving_avg_data[index]

        if (amplitude < threshold):
            cough_indices.remove(index)


    # Finding the timestamps of the coughs
    predicted_timestamps = []
    for index in cough_indices:
        predicted_timestamps.append(round(index / sample_rate, 6))


    # Filtering the sound after coughing
    for ts in predicted_timestamps:
        matches = list((ts_2 for ts_2 in predicted_timestamps if ts < ts_2 < ts + 0.4))

        if len(matches) != 0:
            for i in range(len(matches)):
                index = predicted_timestamps.index(matches[i])
                predicted_timestamps.remove(matches[i])
                cough_indices.remove(cough_indices[index])
    
    # Adding predicted and real timestamp tuples to the list
    
    all_timestamp_data.append((predicted_timestamps, real_timestamps[i_data]))
    
    cough_count = len(cough_indices)

    # The results
    print("\nAuio {}".format(i_data + 1))
    print("---------------------")
    print("Cough Count: {}".format(cough_count))
    print("Cough Predicted Timestamps: {}".format([round(timestamp, 6) for timestamp in predicted_timestamps]))
    print("Cough Indices: {}".format(cough_indices))


Auio 1
---------------------
Cough Count: 8
Cough Predicted Timestamps: [1.459, 6.547917, 7.1175, 12.20125, 21.071521, 29.919354, 39.99475, 50.774188]
Cough Indices: [70032, 314300, 341640, 585660, 1011433, 1436129, 1919748, 2437161]

Auio 2
---------------------
Cough Count: 8
Cough Predicted Timestamps: [4.037104, 6.868521, 10.603729, 23.179312, 27.485542, 39.477854, 45.191646, 56.298688]
Cough Indices: [193781, 329689, 508979, 1112607, 1319306, 1894937, 2169199, 2702337]

Auio 3
---------------------
Cough Count: 4
Cough Predicted Timestamps: [12.6905, 23.158646, 43.378396, 56.403229]
Cough Indices: [609144, 1111615, 2082163, 2707355]

Auio 4
---------------------
Cough Count: 4
Cough Predicted Timestamps: [2.488438, 11.948104, 39.242271, 51.676583]
Cough Indices: [119445, 573509, 1883629, 2480476]

Auio 5
---------------------
Cough Count: 3
Cough Predicted Timestamps: [7.280979, 32.6925, 49.692167]
Cough Indices: [349487, 1569240, 2385224]

Auio 6
---------------------
Cough Coun

In [111]:
# Comparing the real timestamps and predicted timestamps.

def check_performance(predicted_timestamps, real_timestamps):
    time_margin = 0.05 # In seconds
    
    true_positive = 0 
    false_positive = 0

    if (len(predicted_timestamps) > len(real_timestamps)):

        for pred in predicted_timestamps:
            match = list((rt for rt in real_timestamps if pred - 0.4 < rt < pred + 0.4))

            if len(match) != 0:
                true_positive += 1
            else:
                false_positive += 1

    else:

        for pred in real_timestamps:
            match = list((rt for rt in predicted_timestamps if pred - 0.4 < rt < pred + 0.4))

            if len(match) != 0:
                true_positive += 1
            else:
                false_positive += 1
    
    return true_positive, false_positive

In [116]:
# General Performance of the Model
performances = []
true_positives = []
false_positives = []

for i, data in enumerate(all_timestamp_data):
    
    predicted_timestamps = data[0]
    real_timestamps = data[1]
    
    true_positive, false_positive = check_performance(predicted_timestamps, real_timestamps)
    
    true_positives.append(true_positive)
    false_positives.append(false_positive)
    
    precision = true_positive / (true_positive + false_positive)
    # Recall is assumed as 1 since the model does not predict the absence of coughs
    recall = 1

    f1_score = round(2 * (precision * recall) / (precision + recall), 3)
    print("Audio {}: {}".format(i+1,f1_score))

    
model_performance = np.sum(true_positives) / np.sum(true_positives + false_positives)
print("\nModel Performance: ", model_performance)

Audio 1: 1.0
Audio 2: 0.933
Audio 3: 1.0
Audio 4: 1.0
Audio 5: 1.0
Audio 6: 0.615
Audio 7: 0.353
Audio 8: 0.0
Audio 9: 1.0

Model Performance:  0.6229508196721312
